In [1]:

import matplotlib
matplotlib.use('Agg')
import os
import glob
import sys
import h5py
import json
import re
import datetime
import optparse
import pprint
import traceback
import time
import skimage
import shutil
import fissa
import cv2
import math
import multiprocessing as mp
import tifffile as tf
import pylab as pl
import numpy as np
import cPickle as pkl
from skimage import img_as_uint
from pipeline.python.utils import natural_keys, hash_file_read_only, load_sparse_mat, print_elapsed_time, hash_file, replace_root, uint16_to_RGB
from pipeline.python.set_trace_params import post_tid_cleanup
from pipeline.python.rois.utils import get_info_from_tiff_dir
from pipeline.python.traces.utils import get_frame_info, get_metric_set
from pipeline.python.paradigm import align_acquisition_events as acq
pp = pprint.PrettyPrinter(indent=4)

In [21]:

def load_RID(session_dir, roi_id):

    roi_dir = os.path.join(session_dir, 'ROIs')
    roidict_path = glob.glob(os.path.join(roi_dir, 'rids_*.json'))[0]
    with open(roidict_path, 'r') as f:
        roidict = json.load(f)
    RID = roidict[roi_id]

    return RID

    
def load_TID(run_dir, trace_id, auto=False):
    run = os.path.split(run_dir)[-1]
    trace_dir = os.path.join(run_dir, 'traces')
    tmp_tid_dir = os.path.join(trace_dir, 'tmp_tids')
    tracedict_path = os.path.join(trace_dir, 'traceids_%s.json' % run)
    try:
        print "Loading params for TRACE SET, id %s" % trace_id
        with open(tracedict_path, 'r') as f:
            tracedict = json.load(f)
        TID = tracedict[trace_id]
        pp.pprint(TID)
    except Exception as e:
        print "No TRACE SET entry exists for specified id: %s" % trace_id
        print "TRACE DIR:", tracedict_path
        print tracedict.keys()
        try:
            print "Checking tmp trace-id dir..."
            if auto is False:
                while True:
                    tmpfns = [t for t in os.listdir(tmp_tid_dir) if t.endswith('json')]
                    for tidx, tidfn in enumerate(tmpfns):
                        print tidx, tidfn
                    userchoice = raw_input("Select IDX of found tmp trace-id to view: ")
                    with open(os.path.join(tmp_tid_dir, tmpfns[int(userchoice)]), 'r') as f:
                        tmpTID = json.load(f)
                    print "Showing tid: %s, %s" % (tmpTID['trace_id'], tmpTID['trace_hash'])
                    pp.pprint(tmpTID)
                    userconfirm = raw_input('Press <Y> to use this trace ID, or <q> to abort: ')
                    if userconfirm == 'Y':
                        TID = tmpTID
                        break
                    elif userconfirm == 'q':
                        break
        except Exception as e:
            traceback.print_exc()
            print "---------------------------------------------------------------"
            print "No tmp trace-ids found either... ABORTING with error:"
            print e
            print "---------------------------------------------------------------"

    return TID

In [3]:
#%%
def masks_to_normed_array(masks):
    '''
    Assumes masks.shape = (d1, d2, nrois)

    Returns:
        maskarray of shape (d, nrois), where d = d1*d2
        values are normalized by size of mask
    '''
    d1, d2 = masks[:,:,0].shape
    d = d1*d2

    nrois = masks.shape[-1]

    masks_arr = np.empty((d, nrois))
    for r in range(nrois):
        masks_arr[:, r] = np.reshape(masks[:,:,r], (d,), order='C') /  len(np.nonzero(masks[:,:,r])[0])

    return masks_arr


#%%
def get_gradient(im) :
    # Calculate the x and y gradients using Sobel operator
    grad_x = cv2.Sobel(im,cv2.CV_32F,1,0,ksize=3)
    grad_y = cv2.Sobel(im,cv2.CV_32F,0,1,ksize=3)

    # Combine the two gradients
    grad = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)
    return grad

#%%
#def uint16_to_RGB(img):
#    im = img.astype(np.float64)/img.max()
#    im = 255 * im
#    im = im.astype(np.uint8)
#    rgb = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
#    return rgb

def plot_warped_rois(ref, sample, masks, masks_aligned, out_fpath='/tmp/aligned_rois.png'):
    refRGB = uint16_to_RGB(ref)
    imRGB = uint16_to_RGB(sample)
    wimRGB = uint16_to_RGB(sample)
    nrois = masks.shape[-1]

    fig = pl.figure(figsize=(15,5))
    fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, hspace=0.1, wspace=0.1)
    ax1 = fig.add_subplot(1,3,1); pl.imshow(refRGB, cmap='gray'); pl.title('ref rois'); pl.axis('off')
    ax2 = fig.add_subplot(1,3,2); pl.imshow(imRGB, cmap='gray'); pl.title('sample, orig rois'); pl.axis('off')
    ax3 = fig.add_subplot(1,3,3); pl.imshow(imRGB, cmap='gray'); pl.title('sample, warped rois'); pl.axis('off')
    for ridx in range(nrois):
        #roinum = ridx + 1
        orig = masks[:,:,ridx].copy().astype('uint8')
        # Draw contour for ORIG rois on reference:
        ret,thresh = cv2.threshold(orig,.5,255,0)
        orig2,contours,hierarchy = cv2.findContours(thresh, 1, 2)
        cv2.drawContours(refRGB, contours, 0, (0,255,0), 1)
        ax1.imshow(refRGB)
        # Draw orig ROIs on sample:
        cv2.drawContours(imRGB, contours, 0, (0,255,0), 1)
        ax2.imshow(imRGB)
        # Draw orig ROIs + warped ROIs on sample (i.e., ref rois warped to match sample)
        alig = masks_aligned[:,:,ridx].copy().astype('uint8')
        ret,thresh = cv2.threshold(alig,.5,255,0)
        aligC,contours2,hierarchy = cv2.findContours(thresh, 1, 2)
        cv2.drawContours(wimRGB, contours, 0, (0,255,0), 1)
        cv2.drawContours(wimRGB, contours2, 0, (255,0,0), 1)
        ax3.imshow(wimRGB)

    #figname = 'aligned_rois.png'
    pl.savefig(out_fpath)
    pl.close()

In [4]:
def create_neuropil_masks(masks, niterations=10):
    # Create kernel for dilating ROIs:
    kernel = np.ones((3,3),masks.dtype)

    nrois = masks.shape[-1]
    np_masks = np.empty(masks.shape, dtype=masks.dtype)

    for ridx in range(nrois):
        rmask = masks[:,:,ridx]
        if niterations==20:
            gap_iterations = 8
        else:
            gap_iterations = 4
        gap = cv2.dilate(rmask, kernel, iterations=gap_iterations)
        dilated = cv2.dilate(rmask, kernel, iterations=niterations)

        # Subtract to get annulus region:
        annulus = (dilated - gap)

        # Get full mask image to subtract overlaps:
        allmasks = np.sum(masks, axis=-1)
        summed = annulus + allmasks
        summed[summed>1] = 0

        # Add annulus back in to make neuropil area = 2, everythign else = 1:
        summed += annulus
        neuropil = summed - allmasks
        np_masks[:,:,ridx] = neuropil.astype('bool')

    return np_masks

In [5]:
def extract_options(options):
    choices_npmethod = ('fissa', 'annulus')
    default_npmethod = 'annulus'

    choices_tracetype = ('raw', 'raw_fissa', 'denoised_nmf', 'np_corrected_fissa', 'neuropil_fissa', 'np_subtracted', 'neuropil')
    default_tracetype = 'raw'

    parser = optparse.OptionParser()

    # PATH opts:
    parser.add_option('-D', '--root', action='store', dest='rootdir', default='/nas/volume1/2photon/data', help='data root dir (root project dir containing all animalids) [default: /nas/volume1/2photon/data, /n/coxfs01/2pdata if --slurm]')
    parser.add_option('-i', '--animalid', action='store', dest='animalid', default='', help='Animal ID')
    parser.add_option('-S', '--session', action='store', dest='session', default='', help='session dir (format: YYYMMDD_ANIMALID')
    parser.add_option('-A', '--acq', action='store', dest='acquisition', default='FOV1', help="acquisition folder (ex: 'FOV1_zoom3x') [default: FOV1]")
    parser.add_option('-R', '--run', action='store', dest='run', default='', help="name of run dir containing tiffs to be processed (ex: gratings_phasemod_run1)")
    parser.add_option('--default', action='store_true', dest='default', default='store_false', help="Use all DEFAULT params, for params not specified by user (no interactive)")
    parser.add_option('--slurm', action='store_true', dest='slurm', default=False, help="set if running as SLURM job on Odyssey")
    parser.add_option('-t', '--trace-id', action='store', dest='trace_id', default='', help="Trace ID for current trace set (created with set_trace_params.py, e.g., traces001, traces020, etc.)")

    parser.add_option('--new', action="store_true",
                      dest="create_new", default=False, help="Set flag to create new output files (/paradigm/parsed_frames.hdf5, roi_trials.hdf5")
    parser.add_option('--append', action="store_true",
                      dest="append_trace_type", default=False, help="Set flag to append non-default trace type to trace structs (e.g., neuropil correction).")

    parser.add_option('--np', type='choice', choices=choices_npmethod, action='store', dest='np_method', default=default_npmethod, help="Method for neuropil correction. Valid choices: %s [default: %s]" % (choices_npmethod, default_npmethod))

    parser.add_option('-N', '--ncores', action="store",
                      dest="ncores", default=2, help="[np-fissa]: N cores to use for FISSA prep and separation [default: 2, 4. If slurm, 1]")

    # Neuropil options:
    parser.add_option('-a', '--halo', action="store",
                      dest="np_niterations", default=10, help="[np-subtract]:  N iterations for ROI dilation when creating annulus for neuropil [default: 10]")
    parser.add_option('-c', '--cfactor', action="store",
                      dest="np_correction_factor", default=0.5, help="[np-subtract]: Correction factor for neuropil subtraction [default: 0.5]")
    parser.add_option('--neuropil', action="store_true",
                      dest="do_neuropil_correction", default=False, help="Set flag to extract neuropil.")
    parser.add_option('--warp', action="store_true",
                      dest="save_warp_images", default=False, help="Set flag to save output plots of warped ROIs (manual warp only).")

    (options, args) = parser.parse_args(options)

    return options

In [129]:
import collections
def update_dict(d, u):
    for k, v in u.iteritems():
        if isinstance(v, collections.Mapping):
            d[k] = update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

In [228]:
def plot_rois(zproj_img, maskarray, save_dir='/tmp', figname='roi_masks.png', 
                      plot_neuropil=False, np_maskarray=None, np_niterations=None):

    d1, d2 = zproj_img.shape
    nrois = maskarray.shape[1] 

    #print "--- Mask array: %i ROIs on %s, %s" % (nrois, curr_file, curr_slice)
    print "--- Mask array: %i ROIs saved to %s" % (nrois, figname)
    fig = pl.figure()
    ax = fig.add_subplot(1,1,1)
    p2, p98 = np.percentile(zproj_img, (2, 99.98))
    avgimg = skimage.exposure.rescale_intensity(zproj_img, in_range=(p2, p98)) #avg *= (1.0/avg.max())
    ax.imshow(avgimg, cmap='gray')

    npmasks_r = None
    if plot_neuropil is True:
        assert np_maskarray is not None, "No NP masks provided!"
        npmasks_r = np.reshape(np_maskarray, (d1, d2, nrois))
        
    masks_r = np.reshape(maskarray, (d1, d2, nrois))
    
    for ridx in range(nrois):
        masktmp = masks_r[:, :, ridx]
        msk = masktmp.copy()
        msk[msk==0] = np.nan

        if np.isnan(masktmp).all():
            ax.text(1, 1, '%i - no mask' % int(ridx+1), fontsize=8, weight='light', color='r')
        else:
            ax.imshow(msk, interpolation='None', alpha=0.5, cmap=pl.cm.Greens_r, vmin=0, vmax=1.0)
            [ys, xs] = np.where(masktmp>0)
            ax.text(xs[int(round(len(xs)/4))], ys[int(round(len(ys)/4))], str(ridx+1), fontsize=8, weight='light', color='w')

        if npmasks_r is not None and plot_neuropil is True:
            masktmp = npmasks_r[:, :, ridx]
            msk = masktmp.copy()
            msk[msk==0] = np.nan
            ax.imshow(msk, interpolation='None', alpha=0.2, cmap=pl.cm.Blues_r)
    ax.axis('off')
    
    pl.savefig(os.path.join(save_dir, figname))
    pl.close()
    

## RoiMask class definition

In [218]:
class RoiMasks():
    def __init__(self, mask_fpath):
        try:
            maskfile = h5py.File(mask_fpath, 'r')
        except Exception as e:
            print "Unable to read specified mask file at: %s" % mask_fpath
        
        self.source_file = mask_fpath
        self.is_3D = maskfile.attrs['is_3D'] in ['True']
        self.roi_id = maskfile.attrs['roi_id'] 
        self.roi_hash = maskfile.attrs['roi_hash'] 
        self.roi_type = maskfile.attrs['roi_type'] 
        self.zproj_type = maskfile.attrs['zproj']
        
        # Check if masks have a single .tif reference:
        if len(maskfile.keys()) == 1:
            self.ref_file = maskfile.keys()[0]
            self.single_reference = True

        # Get signal channel for ROI source:
        self.signal_channel = os.path.split(os.path.split(maskfile[maskfile.keys()[0]]['masks'].attrs['source'])[0])[-1]
        
        # Get slices for which there are ROIs in this set:
        self.roi_slices = sorted([str(s) for s in maskfile[maskfile.keys()[0]]['masks'].keys()], key=natural_keys)

        # Identify tiff source for ROIs:
        rootdir = mask_fpath.split('/%s' % maskfile.attrs['animal'])[0]
        roidict_path = glob.glob(os.path.join(rootdir, maskfile.attrs['animal'], maskfile.attrs['session'], 'ROIs', 'rids_*.json'))[0]
        with open(roidict_path, 'r') as f: roidict = json.load(f);
        self.roi_tiff_src = roidict[self.roi_id]['SRC']

        # Get masks from ROI SOURCE: -- assuming is_slice_format = True (i.e., ROIs are savd by SLICE, not grouped altogether)
#        self.masks = dict((curr_slice, maskfile[self.ref_file]['masks'][curr_slice][:].T.copy()) \
#                                               for curr_slice in maskfile[self.ref_file]['masks'].keys())
        self.source_mask = maskfile[self.ref_file]['masks']['Slice01'][:].T        
        self.masks = {}

    def get_reference_image(self, zproj_type='mean'):
        
        self.ref_img_path = glob.glob('%s_%s_deinterleaved/%s/%s/*.tif' % (self.roi_tiff_src, zproj_type, self.signal_channel, self.ref_file))[0]
        self.reference_img = tf.imread(self.ref_img_path)

        
    def get_masks(self, target_paths, nprocs=4, save_warp_images=True, output_dir='/tmp'):

        print "Getting masks for %i .tif files." % len(target_paths)
        if save_warp_images:
            print "Saving warped ROI mask images to: %s" % output_dir
    
        t_eval_mp = time.time()
        def worker(target_paths_list, out_q, save_warp_images, output_dir):
            """
            Worker function is invoked in a process. 'target_paths_list' is a list of
            files for which to warp reference masks.
            """
            print "Starting!"
            warps = {}
            for target_fpath in target_paths_list:
                fname = os.path.splitext(os.path.split(target_fpath)[-1])[0]
                curr_file = str(re.search('File(\d{3})', target_fpath).group(0))
                warps[curr_file] = {'maskarray': None, 'np_maskarray': None}
                if target_fpath == self.ref_img_path:
                    # Don't need to do anything but normalize array:
                    warps[curr_file]['maskarray'] = masks_to_normed_array(self.masks)
                else:
                    print "... Warping %s to ref." % target_fpath
                    target_img = tf.imread(target_fpath)
                    masks_aligned = self.warp_mask(target_img)
                    warps[curr_file]['maskarray'] = masks_to_normed_array(masks_aligned)
                    if save_warp_images:
                        warp_img_path = os.path.join(output_dir, 'warped_rois_r%s_to_%s.png' % (self.ref_file, fname))
                        plot_warped_rois(self.reference_img, target_img, self.source_mask, masks_aligned, 
                                             out_fpath=warp_img_path)
            out_q.put(warps)
    
        # Each process gets "chunksize' filenames and a queue to put his out-dict into:
        out_q = mp.Queue()
        chunksize = int(math.ceil(len(target_paths) / float(nprocs)))
        procs = []
        for i in range(nprocs):
            p = mp.Process(target=worker,
                           args=(target_paths[chunksize * i:chunksize * (i + 1)],
                                        out_q,
                                        save_warp_images,
                                        output_dir))
            procs.append(p)
            p.start()
    
        # Collect all results into single results dict. We should know how many dicts to expect:
        resultdict = {}
        for i in range(nprocs):
            resultdict.update(out_q.get())
    
        # Wait for all worker processes to finish
        for p in procs:
            print "Finished:", p
            p.join()
        print_elapsed_time(t_eval_mp)
    
        self.masks = resultdict

    def warp_mask(self, target_img, warp_mode=cv2.MOTION_HOMOGRAPHY):

        height, width = self.reference_img.shape
    
        # Define motion model
        # Set the warp matrix to identity.
        if warp_mode == cv2.MOTION_HOMOGRAPHY:
            warp_matrix = np.eye(3, 3, dtype=np.float32)
        else:
            warp_matrix = np.eye(2, 3, dtype=np.float32)
    
        # Set the stopping criteria for the algorithm.
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000,  1e-6)
    
        # Warp REFERENCE image into sample:
        sample = target_img.copy()
        (cc, warp_matrix) = cv2.findTransformECC (get_gradient(sample), get_gradient(self.reference_img), warp_matrix, warp_mode, criteria)
    
        #% Warp masks with warp transform:
        masks_aligned = np.zeros(self.source_mask.shape, dtype=self.source_mask.dtype)
        nrois = self.source_mask.shape[-1]
        for r in xrange(0, nrois):
            masks_aligned[:,:,r] = cv2.warpPerspective (self.source_mask[:,:,r], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    
        return masks_aligned


    def create_neuropil_masks(self, niter):
        for curr_file in self.masks.keys():
            nrois = self.masks[curr_file]['maskarray'].shape[-1]
            d1, d2 = self.reference_img.shape
            m_array = np.reshape(self.masks[curr_file]['maskarray'], (d1, d2, nrois))
            np_masks = create_neuropil_masks(m_array, niterations=niter)
            self.masks[curr_file]['np_maskarray'] = masks_to_normed_array(np_masks)
        self.np_niterations = niter
            
    def plot_rois(self, plot_neuropil=False, output_dir='/tmp'):
        curr_slice = 'Slice01'
        for curr_file in self.masks.keys():
            if plot_neuropil is True:
                figname = 'rois_%s_%s_%s_np_niter%i.png' % (curr_file, curr_slice, self.roi_id, self.np_niterations)
            else:
                figname = 'rois_%s_%s_%s.png' % (curr_file, curr_slice, self.roi_id)

            curr_maskarray = self.masks[curr_file]['maskarray']
            curr_np_maskarray = self.masks[curr_file]['np_maskarray'] if plot_neuropil else None
            np_iters = self.np_niterations if plot_neuropil else None
            plot_rois(self.reference_img, curr_maskarray, 
                          save_dir=output_dir, figname=figname, 
                          plot_neuropil=plot_neuropil,
                          np_maskarray=curr_np_maskarray,
                          np_niterations=np_iters)
        

### Traces class defintiion:

In [223]:
class Traces():
    
    def __init__(self, optsE):
        self.rootdir = optsE.rootdir
        self.animalid = optsE.animalid
        self.session = optsE.session
        self.acquisition = optsE.acquisition
        self.run = optsE.run
        self.trace_id = optsE.trace_id
        
        # Load meta data for current trace extraction:
        run_dir = os.path.join(self.rootdir, self.animalid, self.session, self.acquisition, self.run)
        print run_dir, self.trace_id
        session_dir = os.path.join(self.rootdir, self.animalid, self.session)
        self.TID = load_TID(run_dir, self.trace_id)
        self.meta = {'SI': get_frame_info(run_dir),
                     'RID': load_RID(session_dir, self.TID['PARAMS']['roi_id'])}
        if self.rootdir not in self.meta['RID']['DST']:
            self.meta['RID']['DST'] = replace_root(self.meta['RID']['DST'], self.rootdir, self.animalid, self.session)
            
        self.masks = None
        
        self.nonnegative = '_offset' in self.TID['SRC'] 
        self.do_neuropil_correction = optsE.do_neuropil_correction
        self.np_method = optsE.np_method
        self.np_niterations = optsE.np_niterations
        self.np_correction_factor = optsE.np_correction_factor
                    
    def get_sources(self):
        self.get_tiff_source()
        self.get_mask_source()


    def get_masks(self, new_soma=False, new_neuropil=False, nprocs=4, save_warp_images=False):
        self.soma_masks(nprocs=nprocs, save_warp_images=save_warp_images, create_new=new_soma)
        self.neuropil_masks(np_method=self.np_method, nprocs=nprocs, create_new=new_neuropil)
        
    def plot_rois(self, plot_neuropil=False):            
        mask_figdir = os.path.join(self.TID['DST'], 'figures', 'masks')
        if not os.path.exists(mask_figdir): os.makedirs(mask_figdir);
        self.masks.plot_rois(plot_neuropil=plot_neuropil, output_dir=mask_figdir)

        
    def get_tiff_source(self):
        # Run tiff pre-processing, if nonnegative specified:
        if self.nonnegative and len(glob.glob(os.path.join(self.TID['SRC'], '*.tif')))==0: #os.path.exists(self.TID['tiff_dir']):
            print "Making tif files psuedo unsigned with offset and uint16 conversion..."
            orig_tiff_dir = self.TID['tiff_dir'].split('_offset')[0]
            self.TID['tiff_dir'] = add_offset_convert_uint16(orig_tiff_dir)
            
        # Get list of files in current trace set:
        excluded_tiffs = self.TID['PARAMS']['excluded_tiffs']
        ntiffs = len(glob.glob(os.path.join(self.TID['SRC'], '*.tif')))
        self.filenames = sorted(['File%03d' % int(i+1) for i in range(ntiffs) if 'File%03d' % int(i+1) not in excluded_tiffs], key=natural_keys)


    def get_mask_source(self):
        # Create mask object:
        mask_fpath = os.path.join(self.meta['RID']['DST'], 'masks.hdf5')
        self.masks = RoiMasks(mask_fpath)
        if self.rootdir not in self.masks.roi_tiff_src:
            self.masks.roi_tiff_src = replace_root(self.masks.roi_tiff_src, self.rootdir, self.animalid, self.session)

        # Determine whether the ROI source is the same as the TIFF source:
        if self.rootdir not in self.masks.roi_tiff_src: 
            self.masks.roi_tiff_src = replace_root(self.masks.roi_tiff_src, self.rootdir, self.animalid, self.session)
        if self.rootdir not in self.TID['SRC']:
            self.TID['SRC'] = replace_root(self.TID['SRC'], self.rootdir, self.animalid, self.session)
            self.TID['DST'] = replace_root(self.TID['DST'], self.rootdir, self.animalid, self.session)

        # Get reference image for masks:
        self.masks.get_reference_image()
            
        # Always use the mean image to do the ROI warping:
        if self.masks.roi_tiff_src == self.TID['SRC']:
            print "Extracting traces from ROI source"
            self.masks.matched_sources = True
            self.masks.zproj_source_dir = '%s_mean_deinterleaved/%s' % (self.masks.roi_tiff_src, self.masks.signal_channel)
            self.masks.zproj_ref = self.masks.ref_file
        else:
            print "Extracting traces from ALT run roi src"
            self.masks.matched_sources = False
            # Identify which file was used as reference, assuming tiffs were preprocessed and motion-corrected:
            if 'mcorrected' in self.TID['SRC']:
                # Use processing info to get motion-corrected reference file:
                processed_dir = os.path.split(self.TID['SRC'].split('/mcorrected')[0])[1]
                process_id = os.path.split(processed_dir)[-1]
                pid_fpath = glob.glob(os.path.join(self.rootdir, self.animalid, self.session, self.acquisition, self.run, 'processed', 'pids_*.json'))[0]
                with open(pid_fpath, 'r') as f: pdict = json.load(f)
                self.masks.zproj_ref = 'File%03d' % int(pdict[process_id.split('_')[0]]['PARAMS']['motion']['ref_file'])
            # Get corresponding zproj source dir:
            self.masks.zproj_source_dir = '%s_mean_deinterleaved/%s' % (self.TID['SRC'], self.masks.signal_channel)

        
    def soma_masks(self, nprocs=4, save_warp_images=False, create_new=False):
        mfile = None
        if not create_new:
            try:
                warped_mask_path = glob.glob(os.path.join(self.TID['DST'], 'MASKS.hdf5'))[0]
                mfile = h5py.File(warped_mask_path, 'r')
                file_masks = dict((curr_file, {'maskarray': np.array(mfile[curr_file]['maskarray'][:])}) 
                                            for curr_file in mfile.keys())
                self.masks.masks = update_dict(self.masks.masks, file_masks)
            except Exception as e:
                traceback.print_exc()
                print "Unable to load existing mask array file. Creating new!"
                create_new = True
            finally:
                if mfile is not None:
                    mfile.close()
                
        if create_new:
            target_paths = glob.glob(os.path.join('%s_mean_deinterleaved/%s' % (self.TID['SRC'], 
                                                                                self.masks.signal_channel), 
                                                                                  'File*', '*.tif'))
            # Check if alrady have plotted masks, if not, create new:
            warps_figdir = os.path.join(self.TID['DST'], 'figures', 'warp_results')
            if not os.path.exists(warps_figdir): os.makedirs(warps_figdir);
            if len(glob.glob(os.path.join(warps_figdir, '*.png'))) != len(target_paths)-1:
                save_warp_images = True

            self.masks.get_masks(target_paths, nprocs=nprocs, save_warp_images=save_warp_images, output_dir=warps_figdir)
            self.save_warped_masks('maskarray')
            
            
    def neuropil_masks(self, np_method='annulus', nprocs=4, create_new=False):
        
        if np_method == 'annulus':
            mfile=None
            if not create_new:
                try:
                    print "Loading annulus for neuropil."
                    warped_mask_path = glob.glob(os.path.join(self.TID['DST'], 'MASKS.hdf5'))[0]
                    mfile = h5py.File(warped_mask_path, 'r')

                    file_masks = dict((curr_file, {'np_maskarray': np.array(mfile[curr_file]['np_maskarray'][:])}) 
                                                for curr_file in mfile.keys())
                    self.masks.masks = update_dict(self.masks.masks, file_masks)
                    self.masks.np_niterations = self.np_niterations
                    
                except Exception as e:
                    print traceback.print_exc()
                    print "Unable to load existing NP masks. Creating new!"
                    create_new = True
                finally:
                    if mfile is not None:
                        mfile.close()
                    
            if create_new:
                self.masks.create_neuropil_masks(niter=self.np_niterations)
                self.save_warped_masks('np_maskarray')
            
    def save_warped_masks(self, mask_key):
        warp_mask_path = os.path.join(self.TID['DST'], 'MASKS.hdf5')
        print "Saving intermediary warped masks to: %s" % warp_mask_path
        mfile = None
        try:
            if not os.path.exists(warp_mask_path):
                mfile = h5py.File(warp_mask_path, 'w')
            else:
                mfile = h5py.File(warp_mask_path, 'r+')

            assert mask_key in self.masks.masks['File001'].keys(), "Specified key %s not found!" % mask_key

            for curr_file in self.masks.masks.keys():
                if curr_file not in mfile.keys():
                    filegrp = mfile.create_group(curr_file)
                else:
                    filegrp = mfile[curr_file]
                    
                if mask_key not in filegrp.keys():
                    #print "Creating: %s" % mask_key
                    m = filegrp.create_dataset(mask_key, 
                                           self.masks.masks[curr_file][mask_key].shape, 
                                           self.masks.masks[curr_file][mask_key].dtype)
                else:
                    #print "Overwriting: %s" % mask_key
                    m = filegrp[mask_key]
                m[...] = self.masks.masks[curr_file][mask_key]
        except Exception as e:
            traceback.print_exc()
        finally:
            if mfile is not None:
                mfile.close()

    


#### TEST trace class:

In [220]:
options = ['-D', '/n/coxfs01/2p-data', '-i', 'CE077', '-S', '20180523', '-A', 'FOV1_zoom1x', 
           '-R', 'gratings_run1', '-t', 'traces004', '--neuropil', '-c', 0.7, '-a', 10]
optsE = extract_options(options)
T = Traces(optsE)


T.get_sources()

T.get_masks(new_soma=False, new_neuropil=False, nprocs=4, save_warp_images=False)


T.plot_rois(plot_neuropil=True)

/n/coxfs01/2p-data/CE077/20180523/FOV1_zoom1x/gratings_run1 traces004
Loading params for TRACE SET, id traces004
{   u'DST': u'/mnt/odyssey/CE077/20180523/FOV1_zoom1x/gratings_run1/traces/traces004_4bf731',
    u'PARAMS': {   u'excluded_tiffs': [],
                   u'hashid': u'ebc661',
                   u'nonnegative': False,
                   u'offset_uint16': False,
                   u'rid_hash': u'adfc97',
                   u'roi_id': u'rois001',
                   u'roi_type': u'manual2D_circle',
                   u'signal_channel': 1,
                   u'tiff_source': u'/mnt/odyssey/CE077/20180523/FOV1_zoom1x/gratings_run1/processed/processed001_cb3497/mcorrected_a0def0',
                   u'uint16': False},
    u'SRC': u'/mnt/odyssey/CE077/20180523/FOV1_zoom1x/gratings_run1/processed/processed001_cb3497/mcorrected_a0def0',
    u'trace_hash': u'4bf731',
    u'trace_id': u'traces004',
    u'version': u'0.1.0'}
ORIG ROOT: /n/coxfs01/2p-data/CE077/20180523/ROIs/rois001_adfc

In [210]:
import tables as tb

tb.file._open_files.close_all()

In [211]:
mfile.close()

In [56]:
warp_mask_path = os.path.join(T.TID['DST'], 'MASKS.hdf5')
mfile = h5py.File(warp_mask_path, 'a')
for curr_file in T.masks.masks.keys():
    if curr_file not in mfile.keys():
        filegrp = mfile.create_group(curr_file)
    else:
        filegrp = mfile[curr_file]
        
    m = filegrp.create_dataset('maskarray', T.masks.masks[curr_file]['maskarray'].shape, 
                        T.masks.masks[curr_file]['maskarray'].dtype)
    
    m[...] = T.masks.masks[curr_file]['maskarray']
    
mfile.close()